This notebook uses hierarchical clustering in order to cluster the data by journal. Journals that tend to review the same books will be grouped together. As we will see, these groupings often make an incredible amount of intuitive sense. They capture social cleavages as diverse as region, gemre, profession, relative status, and even (in rare cases) race.

In [1]:
import pandas as pd
import numpy as np

from scipy.spatial.distance import jaccard, pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet

import matplotlib.pyplot as plt

We'll be working with the same data as before. This time, we'll just need the book-level counts.

In [2]:
df = pd.read_csv('../data/processed/book_reviews.tsv', sep='\t', index_col=0)
df.head()

,AB Bookman's Weekly,Publishers Weekly,Esquire,Booklist,Journal of Aesthetics and Art Criticism,International Philosophical Quarterly,Journal of Marketing,Harvard Law Review,Journal of Business Education,Journal of Home Economics,...,Black Warrior Review,Computers and the Humanities,American Arts,Essays on Canadian Writing`,Performing Arts Review,"Journal of Arts Management, Law, and Society","Studio International, Review",Journal of Black Studies,Lone Star Review,Aspen Journal of the Arts
"Classics In The Kitchen || AABERG, Jean",2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Complete Book Of Knitting || ABBEY, Barbara",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"A Locomotive Engineer's Album || ABDILL, George B",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"House Of India Cookbook || ABDULLAH, Syed",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Rockefeller Billions: World's Most Stupendous Fortune || ABELS, Jules",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


As we saw in the previous notebook, there is a fair bit of variation in the total number of reviews published by each journal. We're going to cut off any column that has fewer than 30 reviews, since these journals likely don't have enough information to be meaningfully clustered. This specific cut-off point is pretty arbitrary; it can be tinkered with without muddling the results.

In [4]:
min_reviews = 30
my_cols = df.columns[df.sum() >= min_reviews]
df = df[my_cols]
df.shape

(326774, 420)

We're going to create a big (and I mean *big*) dendrogram that sorts journals such that those with a lot of overlap in the authors they review are closer together.

To do so, we have to pick a metric by which the similarity of two journals will be assesed. I've selected cosine distance because it corrects for uneven magnitudes of vectors. For example, we might think that an author like Ursula Le Guin appears in a specific journal because she writes science fiction. But at the same time, she appears in some journals like Publisher's Weekly simply because those journals publish many thousands of reviews. Cosine distance mitigates this problem.

Then, we create a distance matrix that gives a score comparing the distances of every journal from every other journal. Lower values indicate closer-together journals.

In [5]:
dists = pdist(df.T, metric='cosine') # transpose the matrix, since we're comparing columns rather than rows
square_dists = squareform(dists)
square_dists = pd.DataFrame(square_dists, index=df.columns, columns=df.columns)
square_dists

,AB Bookman's Weekly,Publishers Weekly,Esquire,Booklist,Journal of Aesthetics and Art Criticism,International Philosophical Quarterly,Journal of Marketing,Harvard Law Review,Journal of Business Education,Journal of Home Economics,...,Daedalus,Africa Today,Downbeat,Theological Studies,Biblical Theology Bulletin,Book Collector's Market,Educational Forum,Stereo Review,Threepenny Review,Government Publications Review
AB Bookman's Weekly,0.000000,0.942976,0.987363,0.934794,0.978612,1.000000,0.997874,0.998513,1.000000,0.994280,...,0.998197,1.000000,1.000000,1.000000,1.000000,0.986278,1.000000,1.000000,0.998197,1.00000
Publishers Weekly,0.942976,0.000000,0.910372,0.591494,0.990499,0.998808,0.999154,0.983033,0.999542,0.990134,...,0.992346,0.998739,0.990003,1.000000,1.000000,0.997572,0.997289,0.986940,0.985171,1.00000
Esquire,0.987363,0.910372,0.000000,0.943017,0.996264,1.000000,1.000000,0.994026,1.000000,0.997447,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.995172,1.00000
Booklist,0.934794,0.591494,0.943017,0.000000,0.990655,0.999711,0.996439,0.986016,0.997776,0.983543,...,0.993962,0.998776,0.990293,0.997761,0.998973,0.993518,0.994736,0.987847,0.993497,0.99367
Journal of Aesthetics and Art Criticism,0.978612,0.990499,0.996264,0.990655,0.000000,0.990260,1.000000,0.998388,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.996092,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Book Collector's Market,0.986278,0.997572,1.000000,0.993518,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000
Educational Forum,1.000000,0.997289,1.000000,0.994736,1.000000,1.000000,1.000000,1.000000,1.000000,0.992433,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.00000
Stereo Review,1.000000,0.986940,1.000000,0.987847,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.927956,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.00000
Threepenny Review,0.998197,0.985171,0.995172,0.993497,0.996092,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.00000


Next, we choose a method for evaluating within-cluster distance. SciPy has several different options with their own strengths and weaknesses. I've chosen the "average" option because it gives the highest cophenetic correlation coefficient, a measure of how faithfully a dendrogram preserves the original paired distances in the data. 

In [14]:
Z = linkage(dists, 'average')
c, coph_dists = cophenet(Z, dists)
print(Z.shape)
print(c)

(419, 4)
0.8420461087711247


Now, we'll make a dendrogram showing the results. I've added the journal total next to each journal label, so you can see that cosine similarity did in fact ensure that clusters are not simply composed of similarly-sized review sets.

Warning: the resulting image is very large. 

In [15]:
fig = plt.figure(figsize=(40, 400))
ax = fig.add_subplot(1, 1, 1)
labels = [f'{col}: {df[col].sum()}' for col in df.columns]
dn = dendrogram(Z, 
                ax=ax, 
                labels=labels, 
                color_threshold=0.95*max(Z[:,2]), 
                orientation='right',
                distance_sort='descending'
)

ax.tick_params(axis='x', which='major', labelsize=40)
ax.tick_params(axis='y', which='major', labelsize=20)
ax.set_xlim(xmin=.5)


plt.show()

["AB Bookman's Weekly: 4349",
 'Publishers Weekly: 51001',
 'Esquire: 620',
 'Booklist: 60419',
 'Journal of Aesthetics and Art Criticism: 954',
 'International Philosophical Quarterly: 164',
 'Journal of Marketing: 573',
 'Harvard Law Review: 382',
 'Journal of Business Education: 506',
 'Journal of Home Economics: 224',
 'Social Education: 1446',
 'Library Journal: 77369',
 'Small Press Review: 201',
 'World Literature Today: 6548',
 'American Historical Review: 14637',
 'Encounter: 2300',
 'Geographical Journal: 3130',
 'Journal of Historical Geography: 383',
 'New Statesman: 11774',
 'Times Literary Supplement: 50327',
 'Bibliographical Society of America, Papers: 1634',
 'International Labour Review: 1822',
 "Center for Children's Books, Bulletin: 10372",
 'Christian Science Monitor: 9077',
 'Horn Book Magazine: 6513',
 'Kirkus Reviews: 54167',
 'Saturday Review/World: 434',
 'Top of the News: 1036',
 "Children's Book Review Service: 3429",
 'Childhood Education: 3413',
 'Catholic